In [2]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('./data/bamboo.csv')
data.dropna(inplace=True)
data = data[:10]
data['text'] = data['title'] + ' ' + data['description']
data = data[['text', 'storypoint']]

# 数据划分（80% 训练，20% 验证）
train_size = int(0.8 * len(data))
train_df = data[:train_size]
eval_df = data[train_size:]

# 转换为 Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)

In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Using device: {device}")

In [ ]:
#for name, module in model.named_modules():
#    print(name)

In [ ]:
# LoRA 配置
lora_config = LoraConfig(
    r=8,  # 低秩参数
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # 只对线性层应用LoRA
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS  # 回归任务
)

# 应用 LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # 仅微调少量参数，减少计算


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

# 重命名 'storypoint' 列为 'label'
train_dataset = train_dataset.rename_column("storypoint", "labels")
eval_dataset = eval_dataset.rename_column("storypoint", "labels")

# 保持数据格式
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    output_dir="./qwen2_lora_regression",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,  # LoRA 允许 batch_size 稍大
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none",
    gradient_accumulation_steps=8,  # 累计梯度，相当于 batch_size 变大
    save_total_limit=2,
    dataloader_num_workers=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

torch.cuda.empty_cache()
trainer.train()

In [3]:
torch.cuda.empty_cache()

In [ ]:
trainer.evaluate()